# Bash variables, control structures and arithmetic

local variable: created in function that will no be available outside. declare a variable using `typeset`; decreases computation time 
- "-i": integer only `typeset -i x` 

Declare command `declare`
- "-l": all strings in that variable are lowercase 
- "-u": uppercase
- "-r": cannot assign a new value (read-only; useful to assign a variable then `delcare -r` so nothing changes)
- "-a": indexed array by integers
- "-A": indexed array by strings

When calling array one should use `${var}` 

```
declare -l lstring="ABCdef"
declare -u ustring="ABCdef"
declare -r readonly="A Value"
declare -a Myarray
declare -A Myarray2

echo lstring = $lstring
echo ustring = $ustring
echo readonly = $readonly
readonly="New Value"
Myarray[2]="Second Value"
echo 'Myarray[2]= ' ${Myarray[2]}
Myarray2["hotdog"]="baseball"
echo 'Myarray2[hotdog]= ' ${Myarray2["hotdog"]}
```

## Read command 
Reads a line into a variable or multiple variables based on spaces
`read a b` ...  reads first word into a then the rest into b 
* convenient to use with a "while" loop (`;` to have two functions in one line )
```
while 
    command list 1
do
    command list 
1
# loops while command list 1 succeeds
```

_while loop to read data in (input in last statement)_
```
while 
    read a b 
do 
    echo a is $a b is $b 
done <data_file
```
_Piping into while loop_
pulls apart the output of `ls -l`
```
ls -l | while 
    read a b c d 
    do 
        echo owner is $c
    done
```

### For loops
```
for <var> in <list>
do 
    command list 
done
```
- `seq` command is helpful with for loops; "for num in `seq 1 5`" (bash run the command in backticks first)
- can also generate sequences via braces such as {1..5}
- loops separates per _word_

```
for d in $(<data_file>)
```
loops over space/newline to separate data in data_file. Could also nest functions in parentheses

To loop over files (glob listing in bash)
```
for j in *.c # finds all find which end in .c
```

`nl` command is like `cat` but adds a numbered line
useful way to read into directory details and pull owner/permissions 

```
ls -l /etc | while
     read a b c d
do
     echo owner is $c
done
```



## Bash functions
commonly used to organize code in a shell program

```
function printhello {
    echo Hello
}
printhello 
```
- shell memorizes function like it's a new command 

### Return command
functions return when there are no statements or a return line is declared 
```
function myfunction {
    echo starting
    return
    echo this will not be executed
}
```
Functions output can be captured `hvar=$(printhello)`

### Exit command 
`exit` terminates the shell; program not to exit functions but shut down the shell 
- `exit <VALUE>` sets the exit status represented by $? to <VALUE>
- successful shell scripts will output 0 by default
- to be more precise in errors occuring during the shell process, exit values can be assigned so the user understands the error 

* Functions must also be exported ("-f") if locally defined to new shells 

## Redirection and Pipes
- Processes normal have 3 files open (`0 => stdin 1 => stdout 2 => stderr`)
command > stdout-here
command 2> stderr-here
command < stdin-from-here
command &> file # file gets stdout and stderr from command, file is created or overwritten 

### Pipe 
changes standard input and output
- `command | command2` ... command2 stdin comes from first's stdout 
- `command 2<&1 | command2`... gets stderr and stdout from command. can also use `command |& command2`
- `command >> file`... appends stdout of command to end of chosen file 
- `command &>> file`... appends stdout and stderr to end of chosen file

### Here document 
way to embed the standard input inside a script. Avoids having to create a new file to hold input values
``` 
sort << EOF # sort will read until it finds ROF
1
2
3
EOF
```

### Open and close file descriptors 
- `exec N< myfile` ... opens file descriptor N for reading from myfile 
- `exec N> myfile` ... opens file descriptor N for writing to myfile
- `exec N<> myfile`... opens file descriptor N for reading and writing 
- `exec N>&- myfile` or `exec N<&- myfile` ... closes read or write descriptor N
- `lsof` ... to see what file descriptors are open for a process 

```
exec 7>/tmp/myfile7
lsof -p $$ # $$ is PID: process ID
```

Find commands outputting to new files 

```
echo Just '>' ---------------------------------------
find /etc -name grub >grub.out
echo Doing '2>' ---------------------------------------
find /etc -name grub 2>errs.out
echo Doing '&>' ---------------------------------------
find /etc -name grub &>both.out
```

`cat`.. get standard input from keyboard

```
exec 19<data_file
lsof -p $$
cat 0<&19
exec 7>&1  # save stdout in 7
exec 1>&-  # close stdout
lsof -p $$
cat   # no stdout now
exec 1>&7 # copy 7 back to stdout
cat
```


## Case statement 
logic constructor in bash
```
case $ans in
yes|Yes|Y|y|y.x) echo "will do!" ;;
n*|N*) echo "will not do!" ;;
*) echo "oops!" ;;
esac
```

### If then else statement 
"else" is optional 
```
if 
grep -q important myfile
then
echo myfile has important
else
echo myfile has no important 
stuff
fi
```

### Tests in bash
`test` is a builtin to check various conditions and set the return code with the result
- loops and conditionals often use the result of test
- alternative is to use `(())` or `[[]]`
Examples.. 
if
- `test -f file` # -f checks for file
- `[[ -f bfile ]]`
- `test $x -gt 5` # gt is greater than 

Other test operators to be used in [[ ex1  ex2]]
- `-eq` ... equals 
- `-ne` ... not equal
- `-lt` ... less than use g for greater
- `-ge` ... greater than or equal, can also use l
In parentheses, use the same expressions as in python (`$$` = AND, `||` = OR)

`test -s X` if something is successful and X exists, `-d X` if directory 

Can also test permissions using 

`test -x/-w/-r X`for permissions 


## Arithmetic operators 

many are the same as python but `^` is bitwise exclusive operator
```
n=5
((n++)) # increment up to 6
```

# Chapter 2: Coding challenge

### Local vars 
- Write a bash script that has a function f 
- Creates a local variable A, sets it to 1, then prints its value
- Sets A=2, calls f, then prints A after f returns 

```
#!/bin/bash
function f {
    typeset A # make variable local
    A=1
}
A=2
f
echo after f, A=$A
```

### Loops
- write a bash script that uses a for loop to loop over all of the files in `/usr/bin` and `echo` the name of any one of them that has inside the string "unable to fork"

hint: 
if
    strings $i | grep -q "unable to fork"
    then

```
#!/bin/bash
cd /usr/bin/
for i in *
do
if [[ -f $i ]] # check if i is a file
then
    if
    strings $i | grep -q "unable to fork" # -q to not see output
    then 
        echo $i has \"unable to fork\" in it
    fi
fi
done
```

### Arithmetic 
- Write a shell script that echos a running total of the RSS and SZ columns from the commands: `ps -ly`

hint:   n=1
    ps -ly | while 
    read c1 c2 c3 c4 c5 c6 c7 c9 10
    do 
        if ((n>1)) # skip first line
        ...

```
#!/bin/bash
n=1
ps -ly | while
read c1 c2 c3 c4 c5 c6 c7 c9 c10
do 
    if ((n>1))
    then
        ((rss=rss+c8))
        (( sz=sz+c9 ))
        echo rss=$rss sz=$sz
    fi
((n++))
done
```



